In [1]:
pip install ripser

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load some packages
from matplotlib.text import Text
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

/Users/floyd/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
pd.read_csv(
        'https://www.dropbox.com/scl/fi/ccpg44dt73ga24p45a0u3/collaborations_Zoology_608.csv.gz?rlkey=8kr35ztwfztz36qww7v1fr926&st=xcxd6s6d&dl=1',
        compression='gzip',
    )

,article_id,year,researcher_id
0,pub.1100184428,2018.0,ur.07500112735.28
1,pub.1100184428,2018.0,ur.0775274140.76
2,pub.1014429482,2008.0,ur.012645135765.17
3,pub.1014429482,2008.0,ur.016413706041.84
4,pub.1071171019,1994.0,ur.01216432663.31
...,...,...,...
1134648,pub.1028766873,2008.0,ur.0747722674.67
1134649,pub.1063382447,2016.0,ur.01114402740.30
1134650,pub.1063382447,2016.0,ur.01137537003.00
1134651,pub.1063382447,2016.0,ur.0577704134.35


In [4]:
pd.read_csv(
        'https://www.dropbox.com/scl/fi/0bsgnsgb1rr1s143cgyd1/articles_Zoology_608.csv.gz?rlkey=emlv5fy52dz0d2rnodygjy9pp&st=kehlrig7&dl=1',
        compression = 'gzip',
    ).columns

/var/folders/v9/_js75nzx64lf8__fmfsscg4c0000gn/T/ipykernel_23794/2360218702.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,12,20) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(


Index(['article_id', 'year', 'date', 'doi', 'volume', 'issue', 'pages',
       'title_preferred', 'abstract_preferred', 'journal_title',
       'citations_count', 'metrics_times_cited', 'metrics_recent_citations',
       'metrics_field_citation_ratio', 'metrics_relative_citation_ratio',
       'altmetrics_score', 'nauthors', 'npatents_citing',
       'ncategory_for_l1_codes', 'ncategory_for_l2_codes',
       'pg_abstract_preferred_lang_code', 'pg_abstract_preferred_lang_conf'],
      dtype='object')

In [5]:
# load the data
df = pd.read_csv(
        'https://www.dropbox.com/scl/fi/a1t16rtialcw03n50ffkc/concepts_Zoology_608.csv.gz?rlkey=vjv60sfbhofbgvzfzdkrlurl1&st=ciu77f72&dl=1',
        compression='gzip',
    )

df

,article_id,year,concept,relevance_mean,concept_freq_in_abstract,concept_no,dfreq_in_category_for_2l,dfreq_in_category_for_2l_year
0,pub.1050083629,1984,mfvsg,0.0,1,0,2,1
1,pub.1050083629,1984,svsg,0.0,1,1,1,1
2,pub.1091469564,1992,1972 random parasitoid model,0.0,1,0,1,1
3,pub.1117742847,2001,198697 lobster research cruise,0.0,1,0,1,1
4,pub.1043257859,2001,1986–97 lobster research cruise,0.0,1,0,2,2
...,...,...,...,...,...,...,...,...
18064866,pub.1000238039,2015,’s worker population size,0.0,1,40,1,1
18064867,pub.1005449634,2012,’s workforce swarm fraction depart,0.0,1,42,1,1
18064868,pub.1134463678,2021,’s bad crop pest,0.0,1,57,1,1
18064869,pub.1083920773,2017,’s bad invasive,0.0,1,54,1,1


In [6]:
# config
MIN_RELEVANCE = 0.7
MIN_YEAR = 1920
MAX_YEAR = 2021  # when the data is from
MIN_CONCEPT_FREQ = 0.0001
MAX_CONCEPT_FREQ = 0.001

# relevance filtering
df = df[df['relevance_mean'] >= MIN_RELEVANCE]

# year filtering
df = df[df['year'] >= MIN_YEAR]

# counts
num_articles = df['article_id'].nunique()
concept_freq = df.groupby('concept').transform('size') / num_articles
df = df[(concept_freq >= MIN_CONCEPT_FREQ) & (concept_freq <= MAX_CONCEPT_FREQ)]

# remove columns we dont care about
df = df[['article_id', 'concept', 'year']]

df

,article_id,concept,year
18622,pub.1039451177,akh receptor,2002
18623,pub.1129686695,akh receptor,2020
18624,pub.1007326462,akh receptor,2006
18625,pub.1101015253,akh receptor,2018
49772,pub.1092160858,acromyrmex leaf cut ant,2017
...,...,...,...
18063221,pub.1010904311,γδ t cell,2012
18063223,pub.1039368032,γδ t cell,2014
18063225,pub.1120146611,γδ t cell,2019
18063226,pub.1040144066,γδ t cell,2012


In [7]:
# get the initial publication
concepts = (
        df
            .sort_values('year')  # sort so earliest year is first
            .groupby('concept')
            .agg(
                year=('year', 'min'),
                count=('article_id', 'nunique')
            )
            .reset_index()
    )

# normalize the year
concepts['norm_year'] = (concepts['year'] - MIN_YEAR) / (MAX_YEAR - MIN_YEAR)
concepts['inv_count'] = 1 / concepts['count']

concepts

,concept,year,count,norm_year,inv_count
0,abiotic factor,2013,7,0.920792,0.142857
1,abscisic acid,2010,4,0.891089,0.250000
2,absence of pollinator,2001,11,0.801980,0.090909
3,abundance of aphids,2007,4,0.861386,0.250000
4,abundance of arthropod,1984,6,0.633663,0.166667
...,...,...,...,...,...
4141,zebrafish brain,1991,4,0.702970,0.250000
4142,zebrafish larvae,2010,4,0.891089,0.250000
4143,zinc finger transcription factor,2002,5,0.811881,0.200000
4144,zooplankton biomass,1969,4,0.485149,0.250000


In [8]:
# get all possible edges
edges = df.merge(df, on=['article_id', 'year'], suffixes=['_source', '_target'])

# remove duplicates
edges = edges[edges['concept_source'] < edges['concept_target']]  # remove self links (u - u) and the second occurance (u - v vs v - u)
# edges = edges.sort_values('year').drop_duplicates(subset=['concept_source', 'concept_target']).reset_index(drop=True)
edges = edges.groupby(['concept_source', 'concept_target']).agg(
        year=('year', 'min'),
        count=('article_id', 'nunique')
    ).reset_index()

# nromalize the year
edges['norm_year'] = (edges['year'] - MIN_YEAR) / (MAX_YEAR - MIN_YEAR)
edges['inv_count'] = 1 / edges['count']

edges

,concept_source,concept_target,year,count,norm_year,inv_count
0,abiotic factor,abundance of insect,2015,1,0.940594,1.000000
1,abiotic factor,different insect pollinator,2015,1,0.940594,1.000000
2,abiotic factor,invertebrate community,2013,1,0.920792,1.000000
3,abiotic factor,non bee insect,2015,1,0.940594,1.000000
4,abiotic factor,non target invertebrate,2013,1,0.920792,1.000000
...,...,...,...,...,...,...
22851,worker bee,worker larvae,2020,1,0.990099,1.000000
22852,worker reproduction,worker sterility,2001,1,0.801980,1.000000
22853,worker reproduction,young larvae,2019,2,0.980198,0.500000
22854,xenopus tropicalis,zinc finger transcription factor,2013,1,0.920792,1.000000


###Processing the Crocker###

In [9]:
# initialize the graph
G = nx.Graph()

# add the nodes
G.add_nodes_from([(c, {'norm_year': ny, 'inv_count': ic}) for c, ny, ic in zip(concepts['concept'], concepts['norm_year'], concepts['inv_count'])])

# add the edges
G.add_edges_from([(u, v, {'norm_year': ny, 'inv_count': ic}) for u, v, ny, ic in zip(edges['concept_source'], edges['concept_target'], edges['norm_year'], edges['inv_count'])])

G

In [10]:
import oat_python as oat
from oat_python import rust

In [11]:
def compute_beta_0_at(t, c, adj_year, adj_inv_count):
    max_dim = 1

    y_adj = adj_inv_count.copy()
    y_adj[adj_year > t] = 0
    y_adj.eliminate_zeros()

    factored = rust.FactoredBoundaryMatrixVr(y_adj, max_dim)
    homology = factored.homology(False, False)

    h0 = homology[homology['dimension'] == 0]
    beta_0 = ((h0['birth'].values <= c) & (h0['death'].values > c)).sum()
    return beta_0


In [12]:
def compute_beta_1_at(t, c, adj_year, adj_inv_count):
    max_dim = 1

    y_adj = adj_inv_count.copy()
    y_adj[adj_year > t] = 0
    y_adj.eliminate_zeros()

    factored = rust.FactoredBoundaryMatrixVr(y_adj, max_dim)
    homology = factored.homology(False, False)

    h0 = homology[homology['dimension'] == 1]
    beta_0 = ((h0['birth'].values <= c) & (h0['death'].values > c)).sum()
    return beta_0


In [15]:
#set up
adj_year = nx.adjacency_matrix(G, weight='norm_year')
adj_year.setdiag([d['norm_year'] for _, d in G.nodes(data=True)])
adj_inv_count = nx.adjacency_matrix(G, weight='inv_count')
adj_inv_count.setdiag([d['inv_count'] for _, d in G.nodes(data=True)])
adj_inv_count = adj_inv_count.sorted_indices()

In [17]:
year_min = adj_year.data.min()
year_max = adj_year.data.max()
inv_min = adj_inv_count.data.min()
inv_max = adj_inv_count.data.max()
t_vals = np.linspace(year_min, year_max, num=100)
c_vals = np.linspace(inv_min, inv_max, num=100)
print(year_max,year_min,inv_max,inv_min)

1.0 0.0 1.0 0.02702702702702703


In [18]:
betti_surface = np.zeros((len(t_vals), len(c_vals)))
for i, t in enumerate(t_vals):
    for j, c in enumerate(c_vals):
        try:
            betti_surface[i, j] = compute_beta_0_at(t, c, adj_year, adj_inv_count)
        except Exception:
            betti_surface[i, j] = np.nan

In [19]:
betti_surface1 = np.zeros((len(t_vals), len(c_vals)))
for i, t in enumerate(t_vals):
    for j, c in enumerate(c_vals):
        try:
            betti_surface1[i, j] = compute_beta_1_at(t, c, adj_year, adj_inv_count)
        except Exception:
            betti_surface1[i, j] = np.nan

In [ ]:
import plotly.graph_objects as go
T, C = np.meshgrid(c_vals, t_vals)

fig = go.Figure(data=[go.Surface(z=betti_surface, x=T, y=C, colorscale='Viridis')])

fig.update_layout(
    scene=dict(
        xaxis=dict(title='inverse count', range=[-0.05, 1.05]),
        yaxis=dict(title='norm year', range=[-0.05, 1.05]),
        zaxis=dict(title='beta0'),
    ),
    width=800,
    height=700,
    margin=dict(r=20, b=10, l=10, t=10)
)

fig.show()

In [ ]:
T, C = np.meshgrid(c_vals, t_vals)

fig = go.Figure(data=[go.Surface(z=betti_surface1, x=T, y=C, colorscale='Viridis')])

fig.update_layout(
    scene=dict(
        xaxis=dict(title='inverse count', range=[-0.05, 1.05]),
        yaxis=dict(title='norm year', range=[-0.05, 1.05]),
        zaxis=dict(title='beta0'),
    ),
    width=800,
    height=700,
    margin=dict(r=20, b=10, l=10, t=10)
)

fig.show()